In [1]:
from pyspark.sql.types import StructType, LongType,StringType, IntegerType, DoubleType

movieSchema = StructType()\
         .add("movieId", IntegerType(), True)\
         .add("title", StringType(), True)\
         .add("genres", StringType(), True)\
         
movieDf = spark.read.format("csv")\
          .option("header", True)\
          .schema(movieSchema)\
          .load("s3://gk-movielens2/movies.csv")

movieDf.show(2)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
41,application_1635142475032_0042,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
|      2|  Jumanji (1995)|Adventure|Childre...|
+-------+----------------+--------------------+
only showing top 2 rows

In [2]:
ratingSchema = StructType()\
         .add("userId", IntegerType(), True)\
         .add("movieId", IntegerType(), True)\
         .add("rating", DoubleType(), True)\
         .add("timestamp", StringType(), True)


ratingDf = spark.read.format("csv")\
          .option("header", True)\
          .schema(ratingSchema)\
          .load("s3://gk-movielens2/ratings.csv")

ratingDf.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
+------+-------+------+---------+
only showing top 2 rows

In [ ]:
# DF - Python
# DF - SQL
# Spark SQL is core the core engine, the code in Python/R/Java/Scala/SQL data frame code [not RDD code] shall be internally converted 
# into Spark SQL engine Catalyst 
# it optimize your sql/execution flow
# if we write the code inefficiently for example, sort and then filter, internally spark DF [not RDD] shall optimize your code and flow
# it mean, it will rewrite your query plan to fitler first and then sort it.

# SPARK SQL, Queries, Database [meta data ] 


In [4]:
# Run spark SQL statement
spark.sql ('SELECT 1 + 1').show()
spark.sql ('SELECT 1 + 1 as result').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|(1 + 1)|
+-------+
|      2|
+-------+

+------+
|result|
+------+
|     2|
+------+

In [7]:
# SPARK DataBase
# Spark Tables/Views types
# 1. Global Temp View/Table. Non persisted, in memory data frame [internally it is a dataframe, in depth with in spark, it is DataSet]
#     SHARED Across multiple SPARK SESSIONS 
# 2. Temp View/Table, non persisted, in memory data frame, it is spark single SPARK SESSION
# 3. Persisted/permanent table, the data and table are persisted, it will be available when we restart/rerun the spark

# behind all spark temp views/table, permanent tables, we have dataframe

# 'default' is the default database name
spark.sql('SHOW TABLES in default').show()

# create temp view from dataframe
movieDf.createOrReplaceTempView('movies')
ratingDf.createOrReplaceTempView('ratings')


# now we can see a table/view called movies
spark.sql('SHOW TABLES in default').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |   movies|       true|
+--------+---------+-----------+

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |   movies|       true|
|        |  ratings|       true|
+--------+---------+-----------+

In [8]:
# sql returns dataframe, immutable
df2 = spark.sql('SELECT * from movies')
df2.printSchema()
df2.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
|      2|  Jumanji (1995)|Adventure|Childre...|
+-------+----------------+--------------------+
only showing top 2 rows

In [10]:
spark.sql('SELECT movieId, upper(title) from movies').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
|movieId|        upper(title)|
+-------+--------------------+
|      1|    TOY STORY (1995)|
|      2|      JUMANJI (1995)|
|      3|GRUMPIER OLD MEN ...|
|      4|WAITING TO EXHALE...|
|      5|FATHER OF THE BRI...|
+-------+--------------------+
only showing top 5 rows

In [11]:
spark.sql('SELECT * from ratings WHERE rating >=3 and rating <= 4 ORDER BY rating DESC').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|    235|   4.0|964980908|
|     1|    480|   4.0|964982346|
|     1|    349|   4.0|964982563|
|     1|      3|   4.0|964981247|
|     1|    356|   4.0|964980962|
+------+-------+------+---------+
only showing top 5 rows

In [19]:
# create a view as CREATE VIEW AS SELECT   STATEMENT
# this create a temp view called popular_movies
spark.sql("""
CREATE OR REPLACE TEMP VIEW popular_movies AS
SELECT movieId,  avg(rating) as avg_rating, count(userId) as total_ratings FROM ratings    
GROUP BY movieId
HAVING  avg_rating >= 3 AND total_ratings >= 100
ORDER BY total_ratings DESC
""")

#most_popular_movies.show(200)

# spark.sql("select * from joindb.products p INNER JOIN joindb.brands b ON p.brand_id=b.brand_id").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [20]:
spark.sql('SHOW TABLES in default').show()
# check popular_movies in listing
spark.sql("SELECT * FROM popular_movies").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------+-----------+
|database|     tableName|isTemporary|
+--------+--------------+-----------+
|        |        movies|       true|
|        |popular_movies|       true|
|        |       ratings|       true|
+--------+--------------+-----------+

+-------+-----------------+-------------+
|movieId|       avg_rating|total_ratings|
+-------+-----------------+-------------+
|    356|4.164133738601824|          329|
|    318|4.429022082018927|          317|
|    296|4.197068403908795|          307|
|    593|4.161290322580645|          279|
|   2571|4.192446043165468|          278|
+-------+-----------------+-------------+
only showing top 5 rows

In [25]:
spark.sql("""
CREATE  OR REPLACE TEMP VIEW most_popular_movies AS
SELECT pm.movieId, avg_rating, total_ratings, title from popular_movies pm  INNER JOIN movies m ON pm.movieId = m.movieId
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [26]:
# check popular_movies in listing
spark.sql("SHOW TABLES IN default").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+-----------+
|database|          tableName|isTemporary|
+--------+-------------------+-----------+
|        |most_popular_movies|       true|
|        |             movies|       true|
|        |     popular_movies|       true|
|        |            ratings|       true|
+--------+-------------------+-----------+

In [27]:
spark.sql("select * FROM most_popular_movies").show(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+-------------+--------------------+
|movieId|        avg_rating|total_ratings|               title|
+-------+------------------+-------------+--------------------+
|    356| 4.164133738601824|          329| Forrest Gump (1994)|
|    318| 4.429022082018927|          317|Shawshank Redempt...|
|    296| 4.197068403908795|          307| Pulp Fiction (1994)|
|    593| 4.161290322580645|          279|Silence of the La...|
|   2571| 4.192446043165468|          278|  Matrix, The (1999)|
|    260| 4.231075697211155|          251|Star Wars: Episod...|
|    480|              3.75|          238|Jurassic Park (1993)|
|    110| 4.031645569620253|          237|   Braveheart (1995)|
|    589| 3.970982142857143|          224|Terminator 2: Jud...|
|    527|             4.225|          220|Schindler's List ...|
|   2959| 4.272935779816514|          218|   Fight Club (1999)|
|      1|3.9209302325581397|          215|    Toy Story (1995)|
|   1196|4.2156398104265405|          21